In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings
import requests
warnings.filterwarnings('ignore')

In [2]:
def get_championship_standing(season):
    URL = 'https://www.racing-statistics.com/en/seasons/' + season
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find('div', {'class': 'blocks blocks2'}).find('table')
    
    table = []
    for row in results.select("tr")[1:]:
        table.append([td.get_text() for td in row.select("td")])
    standings = pd.DataFrame(table,columns=['row_num','remove_0','Driver','remove_1','Team','Points'])
    standings['Rank'] = np.arange(standings.shape[0])+1
    standings = standings.drop(['row_num', 'remove_0', 'remove_1'], axis=1)
    standings['Team'] = standings['Team'].replace(['Alpine F1 Team','Haas F1 Team'],['Alpine','Haas'])
    
    return standings

In [3]:
champ_standings = pd.DataFrame()
champ_standings  = get_championship_standing('2022')
champ_standings.head(21)

,Driver,Team,Points,Rank
0,Max Verstappen,Red Bull,150,1
1,Sergio Pérez,Red Bull,129,2
2,Charles Leclerc,Ferrari,116,3
3,George Russell,Mercedes,99,4
4,Carlos Sainz,Ferrari,83,5
5,Lewis Hamilton,Mercedes,62,6
6,Lando Norris,McLaren,50,7
7,Valtteri Bottas,Alfa Romeo,40,8
8,Esteban Ocon,Alpine,31,9
9,Pierre Gasly,AlphaTauri,16,10


In [5]:
def get_grand_prix_results(season):
    URL = 'https://www.racing-statistics.com/en/seasons/' + season
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find('div', {'class': 'blocks blocks'}).find('table')
    
    table = []
    for row in results.select("tr")[1:]:
        table.append([td.get_text() for td in row.select("td")])
    gp_result = pd.DataFrame(table,columns=['remove_0','Race Date','Grand Prix','Circuit','remove_1','Driver','Team','# of Laps','Lap Time'])
    gp_result = gp_result.drop(['remove_0', 'remove_1'], axis=1)
    
    return gp_result

In [6]:
grand_prix = get_grand_prix_results('2022')
grand_prix.head(8)

,Race Date,Grand Prix,Circuit,Driver,Team,# of Laps,Lap Time
0,2022-03-20,Bahrain Grand Prix,Bahrain International Circuit,Charles Leclerc,Ferrari,57,1:37:33.584
1,2022-03-27,Saudi Arabian Grand Prix,Jeddah Corniche Circuit,Max Verstappen,Red Bull,50,1:24:19.293
2,2022-04-10,Australian Grand Prix,Albert Park Grand Prix Circuit,Charles Leclerc,Ferrari,58,1:27:46.548
3,2022-04-24,Emilia Romagna Grand Prix,Autodromo Enzo e Dino Ferrari,Max Verstappen,Red Bull,63,1:32:07.986
4,2022-05-08,Miami Grand Prix,Miami International Autodrome,Max Verstappen,Red Bull,57,1:34:24.258
5,2022-05-22,Spanish Grand Prix,Circuit de Barcelona-Catalunya,Max Verstappen,Red Bull,66,1:37:20.475
6,2022-05-29,Monaco Grand Prix,Circuit de Monaco,Sergio Pérez,Red Bull,64,1:56:30.265
7,2022-06-12,Azerbaijan Grand Prix,Baku City Circuit,Max Verstappen,Red Bull,51,1:34:05.941


In [8]:
def get_constructor_standing(champ_standings):
    constructor = champ_standings.copy()
    constructor["Points"] = pd.to_numeric(constructor["Points"])
    constructor = constructor.groupby("Team").sum()
    constructor = constructor.drop(['Rank'], axis=1)
    constructor.sort_values("Points",ascending=False,inplace=True) 
    
    return constructor

In [9]:
constructor = get_constructor_standing(champ_standings)
constructor.reset_index()

,Team,Points
0,Red Bull,279
1,Ferrari,199
2,Mercedes,161
3,McLaren,65
4,Alpine,47
5,Alfa Romeo,41
6,AlphaTauri,27
7,Aston Martin,15
8,Haas,15
9,Williams,3


In [10]:
circuit = pd.read_csv('f1_circuit.csv')
circuit.head(22)

,Order,Date,Grand Prix,Circuit,Circuit Length,Number of Laps,Number of Turns,Latitude,Longitude
0,1,2022-03-20,Bahrain Grand Prix,Bahrain International Circuit,5.412,57,15,26.031226,50.513219
1,2,2022-03-27,Saudi Arabian Grand Prix,Jeddah Corniche Circuit,6.174,50,27,21.636631,39.103057
2,3,2022-04-10,Australian Grand Prix,Albert Park Grand Prix Circuit,5.278,58,14,-37.850709,144.969913
3,4,2022-04-24,Emilia Romagna Grand Prix,Autodromo Enzo e Dino Ferrari,4.909,63,19,44.344358,11.713879
4,5,2022-05-08,Miami Grand Prix,Miami International Autodrome,5.412,57,19,25.956623,-80.231395
5,6,2022-05-22,Spanish Grand Prix,Circuit de Barcelona-Catalunya,4.675,66,16,41.569856,2.260096
6,7,2022-05-29,Monaco Grand Prix,Circuit de Monaco,3.337,78,19,43.735520,7.421821
7,8,2022-06-12,Azerbaijan Grand Prix,Baku City Circuit,6.003,51,20,40.372681,49.853140
8,9,2022-06-19,Canadian Grand Prix,Circuit Gilles Villeneuve,4.361,70,14,45.502464,-73.528083
9,10,2022-07-03,British Grand Prix,Silverstone Circuit,5.891,52,18,52.073574,-1.015170
